In [1]:
!nvidia-smi

Mon Mar 15 16:59:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip3 install wandb

In [3]:
from wandb.keras import WandbCallback
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import numpy as np
import wandb
path = '/content/drive/MyDrive/Colab Notebooks/Projects/Covid19-2'

In [5]:
def runtime_prob():
  for _ in range(100000):
    print(_)

In [6]:
data = pd.read_csv(f'{path}/covid_19_data.csv')

In [7]:
# data.head(1)

In [8]:
data.drop(['SNo','Province/State','Last Update'],inplace=True,axis=1)

In [9]:
data.isna().sum()

ObservationDate    0
Country/Region     0
Confirmed          0
Deaths             0
Recovered          0
dtype: int64

In [10]:
data.rename({'ObservationDate':'Date','Country/Region':'Country'},inplace=True,axis=1)

In [11]:
data.dtypes

Date          object
Country       object
Confirmed    float64
Deaths       float64
Recovered    float64
dtype: object

In [12]:
len(data)

236017

In [13]:
data.dropna(inplace=True)

In [14]:
len(data)

236017

In [15]:
import re
dates = []
for date in data['Date']:
  dates.append(int(re.sub('/','',str(date))))

In [16]:
len(dates)

236017

In [17]:
data['Date'] = dates

In [18]:
# data.head()

In [19]:
data.dtypes

Date           int64
Country       object
Confirmed    float64
Deaths       float64
Recovered    float64
dtype: object

In [20]:
big_data,data = train_test_split(data,test_size=0.01)

In [21]:
len(data),len(big_data)

(2361, 233656)

In [22]:
ct = make_column_transformer(
    (MinMaxScaler(),['Date']),
    (OneHotEncoder(),['Country'])
)
X = data.drop(['Confirmed','Deaths','Recovered'],axis=1)
y = data.drop(['Date','Country'],axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)
ct.fit(X)
X_train = ct.transform(X_train)
X_test = ct.transform(X_test)

In [23]:
class Experimentation:
  def experiment(self,amount_of_layers:int,epochs:int,lr:int,optimizer_,neurons_per_layer:int,activation_types:list,metrics_:list,name:str,project_name:str):
    wandb.init(project=project_name,name=name)
    model = tf.keras.Sequential()
    if int(amount_of_layers) != len(activation_types):
      activation_types_new = []
      for _ in range(amount_of_layers):
        activation_types_new.append(activation_types[0])
    for activation_type in activation_types:
      model.add(tf.keras.layers.Dense(neurons_per_layer,activation=activation_type))
    model.add(tf.keras.layers.Dense(1))
    model.compile(loss=tf.keras.losses.mae,metrics=metrics_,optimizer=optimizer_(lr=lr)) # change
    model.fit(X_train,tf.constant(y_train),validation_data=(X_test,tf.constant(y_test)),epochs=epochs,callbacks=[WandbCallback()]) # callbacks # change
    return True

In [24]:
# What are the stuff I am going to try.
# Layers = 1,2,3,4,5,10,15,20,25
# Epochs = 25,50,75,100,125,250,500
# learning rate = 1.0,0.1,0.01,0.001
# total neurons = 25000,50000,75000,100000
# activation type = Sigmoid linear softmax Swish ReLU Tanh

In [25]:
# Epochs

In [26]:
# epoch_types = [25,50,75,100,125,250,500]
# for epoch_type in epoch_types:
#   E = Experimentation()
#   E.experiment(amount_of_layers=1,epochs=epoch_type,lr=0.001,optimizer_=tf.keras.optimizers.Adam,neurons_per_layer=1,activation_types=[None],metrics_=['mse','mae'],name=f'1-Layer-{epoch_type}-Epochs-Testing-1',project_name='Covid19-2-Test-0')

In [27]:
# Layers

In [28]:
# layer_types = [1,2,3,4,5,10,15,20,25]
# for layer_type in layer_types:
#   E = Experimentation()
#   E.experiment(amount_of_layers=layer_type,epochs=50,lr=0.001,optimizer_=tf.keras.optimizers.Adam,neurons_per_layer=1,activation_types=[None],metrics_=['mse','mae'],name=f'{layer_type}-Layer-50-Epochs-Testing-1',project_name='Covid19-2-Test-0')

In [29]:
# lr

In [30]:
# learning_rates = [1.0,0.1,0.01,0.001]
# for learning_rate in learning_rates:
#   E = Experimentation()
#   E.experiment(amount_of_layers=1,epochs=50,lr=learning_rate,optimizer_=tf.keras.optimizers.Adam,neurons_per_layer=1,activation_types=[None],metrics_=['mse','mae'],name=f'New-1-Layer-50-Epochs-learning_rates-lr-{learning_rate}-Testing-1',project_name='Covid19-2-Test-0')

In [31]:
# tn

In [32]:
# neurons = [25000,50000,75000,100000]
# for neuron in neurons:
#   E = Experimentation()
#   E.experiment(amount_of_layers=1,epochs=50,lr=0.001,optimizer_=tf.keras.optimizers.Adam,neurons_per_layer=neuron,activation_types=[None],metrics_=['mse','mae'],name=f'-New-{neuron}-1-Layer-50-Epochs-learning_rates-tn-Testing-1',project_name='Covid19-2-Test-0')

In [33]:
# activation type

In [34]:
# activation_types = ['linear','swish','relu','tanh']
# for activation_type in activation_types:
#   E = Experimentation()
#   E.experiment(amount_of_layers=1,epochs=50,lr=0.001,optimizer_=tf.keras.optimizers.Adam,neurons_per_layer=neuron,activation_types=[activation_type],metrics_=['mse','mae'],name=f'-New-{activation_type}-1-Layer-50-Epochs-activation_type-ac-Testing-1',project_name='Covid19-2-Test-0')

In [35]:
# What are the stuff I am going to try.
# Layers = 2 in 1 output
# Epochs = 500 / 750
# learning rate = 1.0
# total neurons = 100000
# activation type = relu (Sigmoid linear softmax Swish ReLU Tanh)

In [36]:
# # Epochs = 500 or 750 ? 
# E = Experimentation()
# E.experiment(amount_of_layers=2,epochs=500,lr=1.0,optimizer_=tf.keras.optimizers.Adam,neurons_per_layer=100000,activation_types=['relu'],metrics_=['mse','mae'],project_name='Covid19-2-Test-1',name='Epochs = 500 or 750 ? (500)')
# E.experiment(amount_of_layers=2,epochs=750,lr=1.0,optimizer_=tf.keras.optimizers.Adam,neurons_per_layer=100000,activation_types=['relu'],metrics_=['mse','mae'],project_name='Covid19-2-Test-1',name='Epochs = 500 or 750 ? (750)')

In [37]:
# E.experiment(amount_of_layers=2,epochs=1000,lr=1.0,optimizer_=tf.keras.optimizers.Adam,neurons_per_layer=100000,activation_types=['relu'],metrics_=['mse','mae'],project_name='Covid19-2-Test-1',name='1000-Epochs')
# E.experiment(amount_of_layers=2,epochs=1250,lr=1.0,optimizer_=tf.keras.optimizers.Adam,neurons_per_layer=100000,activation_types=['relu'],metrics_=['mse','mae'],project_name='Covid19-2-Test-1',name='1000-Epochs')

In [41]:
# What are the stuff I am going to try.
# Layers = 2 in 1 output
# Epochs = 275
# learning rate = 1.0
# total neurons = 100000
# activation type = relu (Sigmoid linear softmax Swish ReLU Tanh)

In [42]:
E.experiment(amount_of_layers=2,epochs=275,lr=1.0,optimizer_=tf.keras.optimizers.Adam,neurons_per_layer=100000,activation_types=['relu'],metrics_=['mse','mae'],project_name='Covid19-2-Test-1',name='1% Big-Dog')

NameError: ignored